# Construction and meshing of fracture networks

In this tutorial, we will show:

1. How to define fractures and a fracture network in a 3d domain.
2. How to construct a family of meshes that represent the 3d domain, the fractures and their intersections.
3. Assembly of the grids into a `GridBucket` container that stores all grids, and the geometric relation between them.

Together, these are the first steps towards creating a simulation model for a mixed-dimensional problem in fractured domains.


## Grid bucket directly from fractures
For most simulation purposes, the final grid bucket is all that is needed. Therefore, we start by showing a shortcut for obtaining a `GridBucket` given a set of fractures, a domain and two mesh size parameters. All these will be described in more detail below.

In [ ]:
import numpy as np
import porepy as pp

mesh_size_frac = 0.7
mesh_size_min = 0.2
domain = {'xmin': -2, 'xmax': 3, 'ymin': -2, 'ymax': 3, 'zmin': -3, 'zmax': 3}
f_1 = np.array([[0, 1, 2, 0], [0, 0, 1, 1], [0, 0, 1, 1]])
f_2 = np.array([[0.5, 0.5, 0.5, 0.5], [-1, 2, 2, -1], [-1, -1, 2, 2]])
gb = pp.meshing.simplex_grid([f_1, f_2], domain, mesh_size_frac=mesh_size_frac, mesh_size_min=mesh_size_min, ensure_matching_face_cell=False)

 Note the option ensure_matching_face_cell=False - this is needed to circumvent some issues with the way gmsh produces grids. As far as we are aware, it has no consequences for the simulation quality, but we are working on fixing it.
 
We will now investigate the process carried out by simplex_grid step by step in order to highlight the inner workings of the meshing algorithm. 

# Fractures and a fracture network

Functionality for fractures and their intersection are provided in the subpackage `porepy.fracs`. Fractures are defined either as Elliptic fractures, or as convex, planar polygons.

In [1]:
# Specify a fracture by its vertexes, as a 3xn array
f_1 = pp.Fracture(f_1)

# .. and another fracture, intersecting the first
f_2 = pp.Fracture(f_2)
               

We can also specify the fracture as an ellipsis, approximated as a polygon.

In [2]:
# Specify the fracture center
center = np.array([0.1, 0.3, 0.2])
# The minor and major axis
major_axis = 1.5
minor_axis = 0.5

# Rotate the major axis around the center.
# Note that the angle is measured in radians
major_axis_angle = np.pi/6

# So far, the fracture is located in the xy-plane. To define the incline, specify the strike angle, and the dip angle.
# Note that the dip rotation is carried out after the major_axis rotation (recall rotations are non-commutative).
strike_angle = -np.pi/3
dip_angle = -np.pi/3

# Finally, the number of points used to approximate the ellipsis. 
# This is the only optional parameter; if not specified, 16 points will be used.
num_pt = 12
f_3 = pp.EllipticFracture(center, major_axis, minor_axis, major_axis_angle, strike_angle, dip_angle, num_points=num_pt)

The fractures can be joined into a `FractureNetwork`.

In [18]:
network = pp.FractureNetwork([f_1, f_2, f_3])

The `FractureNetwork` class is the base for analysis and manipulation of fracture networks. The functionality is expanding on demand. For the moment, the most interesting feature is the export of the fracture network to ParaView (requires the vtk extension of python installed, see installation instruction):

In [19]:
network.to_vtk('fracture_network.vtu')

The resulting file can be opened in ParaView. A little bit of work in ParaView gives the following picture

<img src="fracture_network.png" width=300>

We have not yet set a boundary for the `FractureNetwork`, and effectively for the domain. The boundary is defined as a box, and is imposed in the following way

In [20]:
# The domain is a dictionary with fields xmin, xmax, etc.
domain = {'xmin': -2, 'xmax': 3, 'ymin': -2, 'ymax': 3, 'zmin': -3, 'zmax': 3}
network.impose_external_boundary(domain)

Above, we defined the bounding box to not intersect with the fractures. If the domain would have been smaller, fractures that intersect a face of the box would by default (can be overruled) have been truncated so that they are confined within the bounding box.

# Meshing

Our aim is to create a computational mesh that conforms to the fractures, as well as to their intersections (1d lines, 0d points). For the actual grid construction we rely on Gmsh (Triangle / Tetgen will be added in the summer of 2017). However, these packages all require that the geometric constraints, that is the fractures, are described as *non-intersecting* polygons [if you know of packages that do not require this, please let us know]. It only takes some thinking to understand why the meshing software would not like to do this themselves; this is a highly challenging task.

PorePy provides functionality for finding intersections between fractures, and splitting them into non-intersecting polygons. Intersections are found by 

In [21]:
network.find_intersections()

To get information on the number of intersections, type

In [22]:
network.intersection_info()

In total 9 fractures intersect in 15 intersections


When we have found all intersections, the fracture planes should be split into polygons that do not intersect, but that may share edges along intersection lines.

In [23]:
network.split_intersections()

### Geometric tolerances and stability of meshing algorithm
A critical concept in meshing of fractured domains is the concept of geometric tolerance: Since we are operating in finite precision aritmethics, two points may or may not be consider equal (or similarly, two lines / planes may or may not intersect), depending on how accurately we consider their representation. At least three concepts come into play here

1. The accuracy of the numerical representation of the objects (accumulated effect of finite precision rounding errors).
2. The accuracy in geological interpretation of fracture data: If the fracture network originates from an interpretation of satellite images, differences measured in centimeters should be treated with some caution
3. The resolution of the computational grid: If points with a certain distance are considered non-equal, this may also require that we resolve their difference in the mesh. In addition, the mesh generator will use its own concept of geometric tolerance for internal calculations.

In PorePy, these issues are attempted resolved as follows: The `FractureNetwork` has an attribute `tol` that represent the geometric tolerance used in the calculation of intersections and subsequent splitting of the fractures. If meshing is done with gmsh, the tolerances used in PorePy and gmsh are related. The approach works reasonably well, but for complex configurations of fracture intersections, stability issues can arise. We are working to iprove these matters.

## Interaction with gmsh

Next, create grids for the domain, as well as for fractures and fracture intersections. This involves creating a config file for the mesh generator that contains geometry description, including fracture planes and their intersections. The mesh is then created by calling gmsh (NOTE: The path to the gmsh executable should be specified in a PorePy config file, type 'porepy.utils.read_config?' for more information). The resuling mesh information is read back to python, and `Grid` objects representing the matrix, fractures and fracture intersections are created.



Gmsh is quite flexible in terms of letting the user set / guide the preferred mesh size in different parts of the domain. PorePy tries to adjust to this adapting the specified mesh size to the geometry. From the user side, two parameters must be specified: mesh_size_frac gives the target mesh size in the absence of geometric constraints, while mesh_size_min gives the minimal mesh size to be specified to Gmsh. What actually happens with the mesh, that is, how Gmsh translates these preferred options into a grid, is another matter. It may take some practice to get this to work properly.


In [24]:
mesh_size_frac = 0.7
mesh_size_min = 0.2

# Create the grids
grids = pp.fracs.simplex.tetrahedral_grid(network=network, mesh_size_frac=mesh_size_frac, mesh_size_min=mesh_size_min, verbose=1)

Use existing intersections
Use existing decomposition
Minimal distance between points encountered is 0.0659160091619
Gmsh processed file successfully


Grid creation completed. Elapsed time 0.1311185359954834


Created 1 3-d grids with 7378 cells
Created 3 2-d grids with 365 cells
Created 3 1-d grids with 14 cells




As we see from the output, gmsh has created one 3d grid, 3 2d grids (one per fracture) and 3 1d grids along fracture intersections. 

So far there are no connections between the grids. To do this, we will construct a `GridBucket` from the list of grids. In its core, the `GridBucket` is a graph, where the nodes represent individual grids of the matrix, fractures etc, while the edges are connections between the grids. The `GridBucket` also provides methods to work with the hierarchy of grids.

To create the `GridBucket`, we use

In [25]:
# For 1d lines, we need to differentiate between those along fracture intersections, and those along fracture tips
pp.meshing._tag_faces(grids)

# Assemble the grids in a GridBucket
gb = pp.meshing.assemble_in_bucket(grids)
# Compute geometry of all grids in the bucket
gb.compute_geometry()

Each face in the 3d grid that lies on a fracture will coincide with a cell in the 2d grid. The final step involves to split the face into two, one laying on each side of the fracture, by duplicating the nodes on the fracture surface. Similarly, we split faces of fracture grids that coincide with 1d grids, and 1d faces and 0d grids, respectively.

Also create a mapping between the grids on the graph edge.

In [27]:
# To see the effect of the splitting, consider the number of cells, faces and nodes before and after.
def report_grid(gb):
    # Find the grid 
    g = [g for g, _ in gb if g.dim == gb.dim_max()][0]
    print('Number of cells: ' + str(g.num_cells))
    print('Number of faces: ' + str(g.num_faces))
    print('Number of nodes: ' + str(g.num_nodes))

print('Before splitting')
report_grid(gb)
print('\n')
pp.fracs.split_grid.split_fractures(gb)

print('After splitting')
report_grid(gb)

Before splitting
Number of cells: 7378
Number of faces: 17118
Number of nodes: 2565


After splitting
Number of cells: 7378
Number of faces: 17118
Number of nodes: 2565


### Full GridBucket with shortcut
Remember that we will usually prefer to do all of the above using the shortcut introduced at the very beginning. Seeing as all fractures, the domain and the mesh size parameters are already defined, the full `GridBucket` with all three fractures is constructed by 

In [ ]:
gb = pp.meshing.simplex_grid([f_1, f_2, f_3], domain, mesh_size_frac=mesh_size_frac, mesh_size_min=mesh_size_min, ensure_matching_face_cell=False)

# Visualization of the mixed-dimensional mesh
The set of meshes in the `GridBucket` can be dumped to ParaView by simply writing

In [32]:
e = pp.Exporter(gb, 'grid_bucket')
e.write_vtk()

Again, some manipulations in ParaView show how the grids on fracture surfaces intersects with the matrix grid.

<img src='mixed_dimensional_grid.png'  width=200>



### How about 2d problems?
Working with fracture networks in 2d domains, and in particular meshing, is significantly simpler than in 3d. The main reason is that the possible intersection configurations in 2d is much simpler, basically, lines can meet eithen in an X- or a Y-intersection. For this reason, the FractureNetwork class has not been extended to 2d problems; there has simply not been the use for it.

This is of course not to say that meshing in 2d is not difficult; but the challenges mainly lie in the actual grid construction, which we outsource to external packages. 

# Next steps
Now that we have created the `GridBucket`, the next step is to solve mixed-dimensional flow and transport problems. This is covered by the tutorial Darcy_equation, among others.